# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = "../Weatherpy/output_data/cities.csv"
df = pd.read_csv(file)
df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hilo,19.73,-155.09,82.40,58,75,9.17,US,1607477493
1,1,Rikitea,-23.12,-134.97,77.14,78,2,14.05,PF,1607477666
2,2,Myanaung,18.28,95.32,70.92,74,85,7.29,MM,1607477667
3,3,George Town,5.41,100.34,82.40,78,20,4.70,MY,1607477592
4,4,Atuona,-9.80,-139.03,79.20,73,9,12.82,PF,1607477667


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = df[["Lat", "Lng"]].astype(float)
Humidity = df["Humidity"].astype(float)
fig = gmaps.figure(center=(20, -5.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#filter dataframe to include max temp less than 80F but greater than 70F, wind speed less than 10, and cloudiness of 0
hotel_df = df.loc[(df["Max Temp"] < 80) & (df["Max Temp"] > 70) & (df["Wind Speed"] < 10) & (df["Cloudiness"] == 0)]
#drop NAN values from dataframe
hotel_df = hotel_df.dropna()
#get 10 or less rows
hotel_df = hotel_df.iloc[0:10]
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
50,50,Bang Saphan,11.21,99.51,77.00,94,0,2.24,TH,1607477680
52,52,Qandala,11.47,49.87,76.39,51,0,4.99,SO,1607477680
75,75,Asunción,-25.30,-57.64,77.00,65,0,4.70,PY,1607477551
104,104,Ormara,25.21,64.64,75.76,63,0,4.52,PK,1607477694
133,133,Jiwani,25.05,61.74,73.24,57,0,8.68,PK,1607477701
157,157,Sur,22.57,59.53,71.67,69,0,8.14,OM,1607477709
200,200,Cayenne,4.93,-52.33,73.40,100,0,1.12,GF,1607477722
286,286,Jardim,-21.48,-56.14,73.56,56,0,3.62,BR,1607477586
327,327,Makkah al Mukarramah,21.43,39.83,72.43,36,0,1.81,SA,1607477601
359,359,Banikoara,11.30,2.44,73.31,25,0,2.89,BJ,1607477764


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df["Hotel Name"] = ""
base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"

input1 = "hotel"
inputtype = "textquery"
fields = "photos,formatted_address,name,opening_hours,rating"
location = "circle:5000@"

#iterate through hotel_df
for index, row in hotel_df.iterrows():
    #get lat and lng parameters from df
    lat = row["Lat"]
    lng = row["Lng"]
    locationbias = location + str(lat) + "," + str(lng)
    #set up parameters for json call
    params = {"input":input1,
            "inputtype":inputtype,
            "fields":fields,
            "locationbias":locationbias,
            "key":g_key}
    response = requests.get(base_url, params = params).json()
    hotel_df.loc[index, "Hotel Name"] = response["candidates"][0]["name"]



In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))